# Laboratorio 1
Universidad del Valle de Guatemala<br>
Cifrado de Información<br>
Pablo Andrés Zamora Vásquez<br>
Carné 21780
<br><br>
<hr>

## Parte 1 - Ingeniería de características

### Exploración de datos

In [63]:
# 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones

import pandas as pd

df = pd.read_csv('dataset_phishing.csv')

print(df.head(5))

                                                 url      status
0              http://www.crestonwood.com/router.php  legitimate
1  http://shadetreetechnology.com/V4/validation/a...    phishing
2  https://support-appleld.com.secureupdate.duila...    phishing
3                                 http://rgipt.ac.in  legitimate
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate


In [64]:
# 2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “phishing”. ¿Está balanceado el dataset?

print(df['status'].value_counts())

status
legitimate    5715
phishing      5715
Name: count, dtype: int64


Sí, el dataset está balanceado. Tiene la misma cantidad de registros legítimos y de phishing.

### Derivación de características

1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo
de vida del dominio, o las características de la página Web?

El análisis de una URL presenta varias ventajas al compararlo con el análisis de listas negras ("blacklists"), el análisis de contenido, el análisis de vida útil del dominio o el análisis de similaridades entre sitios maliciosos y sitios legítimos. Entre ellas:

- **Detección en tiempo real:** El análisis de URLs permite identificar sitios de phishing desde el momento de su creación, ya que se basa completamente en el análisis estadístico de la URL de un sitio para determinar si es malicioso. Esto lo hace mucho más efectivo contra amenazas de cero horas ("zero-hour threats"); es decir, aquellas amenazas que no se hayan visto antes y, por lo tanto, no son conocidas por una firma de malware 

- **Independencia de terceros:** Como se mencionó antes, el análisis de URLs es independiente; es decir, no depende de bases de datos externas, como listas negras o servicios de reputación de domino para determinar si un sitio es malicioso o no, lo que lo hace más rápido y autónomo.

- **Menor consumo de recursos:** Dado que este análisis se basa únicamente en las URLs de sitios, no necesita realizar consultas externas, por ejemplo, para obtener el contenido de la página. Esto reduce considerablemente el tiempo de procesamiento.

- **A prueba de técnicas de evasión:** Los responsables de un sitio malicioso pueden modificar fácilmente su contenido para evadir la detección basada en características sospechosas de una página web, pero es más laborioso cambiar los patrones de las URLs sin afectar su funcionalidad.

- **Indiferente del lenguaje:** La efectividad de este análisis no se ve afectada por el idioma del contenido del sitio, ya que únicamente trabaja con la estructura de la URL.

(URL-based Phishing Detection using the Entropy of NonAlphanumeric Characters)<br>
(Machine learning based phishing detection from URLs)
<hr>

2. ¿Qué características de una URL son más prometedoras para la detección de phishing?

Algunas características que podrían ser de utilidad en el análisis y clasificación de URLs son:

- **Longitud de la URL:** Las URLs maliciosas tienden a ser más largas que las URLs legítimas, esto podría deberse a que diferentes DGAs utilizan nombres de dominios legítimos para incluir en la URL generada. Por ejemplo googlexyz.com.

- **Cantidad de caracteres especiales:** Caracteres especiales como "@", "_", "?" suelen presentarse con más frecuencia en URLs maliciosas.

- **Cantidad de subdominios presentes en la URL:** Una cantidad elevada de subdominios presentes en la URL puede ser un indicador de phishing. Por ejemplo: login.bank.website.xyz.com.

- **Uso de direcciones IP en lugar de nombres de dominio:** Las URLs maliciosas a menudo utilizan una IP pública en lugar de un nombre de dominio legible.

- **Número de redirecciones:** Las URLs que indican redirecciones a otros sitios suelen ser sospechosas.

- **Uso de HTTPS:** Muchas direcciones de phishing utilizan encriptado HTTPS para dar confianza al usuario, a pesar de que esto no implica que el uso que haga el sitio de su información se legítimo.

- **Cantidad de directorios en la ruta:** Un número alto de directorios en la URL puede ser un indicador de phishing.

- **Uso de acortadores:** El uso de servicios como *bit.ly* o *tinyurl.com* pueden utilizarse para ocultar la verdadera dirección del sitio.

- **Cantidad de caracteres repetidos:** Una gran cantidad de caracteres repetidos en la URL puede ser sospechoso.

- **Uso de símbolos engañosos:** Caracteres como "@" en una URL pueden indicar intentos de suplantación de identidad o *spoofing*.

- **Cantidad de parámetros en la URL:** Una cantidad excesiva de parámetros en la cadena de consulta puede ser una señal de phishing.

- **Frecuencia de caracteres no-alfanuméricos:** La distribución de caracteres no alfanuméricos puede ser un indicador distintivo entre URLs legítimas y maliciosas.

- **Entropía de Shannon:** La entropía mide el nivel de desorden en la URL, y las URLs de phishing suelen tener una entropía mayor.

- **Entropía relativa:** Permite comparar la entropía de una URL con la entropía promedio de URLs legítimas y maliciosas para mejorar la detección.

(URL-based Phishing Detection using the Entropy of NonAlphanumeric Characters)<br>
(Machine learning based phishing detection from URLs)
<hr>

In [65]:
import re
import math
import urllib.parse
from collections import Counter

def url_length(url):
    return len(url)

def count_special_chars(url):
    return sum(1 for c in url if not c.isalnum())

def count_subdomains(url):
    return url.count('.')

def has_ip_address(url):
    return bool(re.search(r'\d+\.\d+\.\d+\.\d+', url))

def count_redirects(url):
    return url.count('//') - 1

def has_https(url):
    return url.startswith("https://")

def contains_suspicious_words(url):
    suspicious_keywords = ["secure", "account", "login", "verify", "bank", "update"]
    return any(word in url.lower() for word in suspicious_keywords)

def count_directories(url):
    return url.count('/')

def uses_shortener(url):
    shorteners = ["bit.ly", "tinyurl", "goo.gl"]
    return any(shortener in url for shortener in shorteners)

def count_repeated_chars(url):
    return sum(1 for match in re.finditer(r'(.)\1{2,}', url))

def contains_at_symbol(url):
    return '@' in url

def count_query_params(url):
    parsed_url = urllib.parse.urlparse(url)
    return len(urllib.parse.parse_qs(parsed_url.query))

def shannon_entropy(url):
    prob = [ float(url.count(char)) / len(url) for char in dict.fromkeys(list(url)) ] 
    entropy = - sum([ p * math.log2(p) for p in prob ]) 
    return entropy

def relative_entropy(url):
    entropy = 0.0
    length = len(url) * 1.0

    if length > 0:
        cnt = Counter(url)
            
        probabilities = {
            '-': 0.013342298553905901,
            '_': 9.04562613824129e-06,
            '0': 0.0024875471880163543,
            '1': 0.004884638114650296,
            '2': 0.004373560237839663,
            '3': 0.0021136613076357144,
            '4': 0.001625197496170685,
            '5': 0.0013070929769758662,
            '6': 0.0014880054997406921,
            '7': 0.001471421851820583,
            '8': 0.0012663876593537805,
            '9': 0.0010327089841158806,
            'a': 0.07333590631143488,
            'b': 0.04293204925644953,
            'c': 0.027385633133525503,
            'd': 0.02769469202658208,
            'e': 0.07086192756262588,
            'f': 0.01249653250998034,
            'g': 0.038516276096631406,
            'h': 0.024017645001386995,
            'i': 0.060447396668797414,
            'j': 0.007082725266242929,
            'k': 0.01659570875496002,
            'l': 0.05815885325582237,
            'm': 0.033884915513851865,
            'n': 0.04753175014774523,
            'o': 0.09413783122067709,
            'p': 0.042555148167356144,
            'q': 0.0017231917793349655,
            'r': 0.06460084667060655,
            's': 0.07214640647425614,
            't': 0.06447722311338391,
            'u': 0.034792493336388744,
            'v': 0.011637198026847418,
            'w': 0.013318176884203925,
            'x': 0.003170491961453572,
            'y': 0.016381628936354975,
            'z': 0.004715786426736459
        }
            
        for char, count in cnt.items():
            try:
                observed = count / length
                expected = probabilities.get(char, 0.0001)  # Asigna un valor mínimo en lugar de fallar
                entropy += observed * math.log2(observed / expected)
            except:
                print(url)
        return entropy


### Preprocesamiento

In [66]:
# Realice las modificaciones necesarias para convertir la variable categórica status a una variable binaria.

df_features = df.copy()
df_features['status'] = df['status'].apply(lambda x: 1 if x == 'phishing' else 0)

# Agregar las características adicionales al dataset

def extract_domain(url):
    parsed_url = urllib.parse.urlparse(url)
    return parsed_url.netloc

df_features["domain"] = df["url"].apply(extract_domain)

# Características que usan la URL completa

df_features["url_length"] = df_features["url"].apply(url_length)
df_features["special_char_count"] = df_features["url"].apply(count_special_chars)
df_features["redirect_count"] = df_features["url"].apply(count_redirects)
df_features["has_https"] = df_features["url"].apply(has_https)
df_features["contains_suspicious_words"] = df_features["url"].apply(contains_suspicious_words)
df_features["directory_count"] = df_features["url"].apply(count_directories)
df_features["repeated_char_count"] = df_features["url"].apply(count_repeated_chars)
df_features["contains_at_symbol"] = df_features["url"].apply(contains_at_symbol)
df_features["query_param_count"] = df_features["url"].apply(count_query_params)

# Características que usan el dominio
df_features["subdomain_count"] = df_features["domain"].apply(count_subdomains)
df_features["has_ip"] = df_features["domain"].apply(has_ip_address)
df_features["uses_shortener"] = df_features["domain"].apply(uses_shortener)
df_features["domain_entropy"] = df_features["domain"].apply(shannon_entropy)
df_features["relative_entropy"] = df_features["domain"].apply(relative_entropy)

print(df_features['status'].value_counts())
print(df_features.head(5))

status
0    5715
1    5715
Name: count, dtype: int64
                                                 url  status  \
0              http://www.crestonwood.com/router.php       0   
1  http://shadetreetechnology.com/V4/validation/a...       1   
2  https://support-appleld.com.secureupdate.duila...       1   
3                                 http://rgipt.ac.in       0   
4  http://www.iracing.com/tracks/gateway-motorspo...       0   

                                              domain  url_length  \
0                                www.crestonwood.com          37   
1                            shadetreetechnology.com          77   
2  support-appleld.com.secureupdate.duilawyeryork...         126   
3                                        rgipt.ac.in          18   
4                                    www.iracing.com          55   

   special_char_count  redirect_count  has_https  contains_suspicious_words  \
0                   7               0      False                      Fals

In [67]:
# Contar la cantidad de valores nulos en cada columna
df.isnull().sum()

url       0
status    0
dtype: int64

### Selección de características